In [1]:
!git clone https://github.com/arnav10goel/CSE556-NLP-Project.git
%cd CSE556-NLP-Project

Cloning into 'CSE556-NLP-Project'...
remote: Enumerating objects: 431, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 431 (delta 78), reused 55 (delta 36), pack-reused 294 (from 1)
Receiving objects: 100% (431/431), 32.34 MiB | 28.50 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/kaggle/working/CSE556-NLP-Project


In [2]:
import torch
from transformers import BertModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from transformers import BertTokenizer
import string

### Load Dataset for the Particular Language
- Here insert the language initial you want to load the train and test CSVs for
- Language Codes:
    1. Hindi: `hi`
    2. Bengali: `bn`
    3. Marathi: `mr`
    4. Tamil: `ta`
    5. Telugu: `te`

In [3]:
lang = 'te'
train_file = f"/kaggle/working/CSE556-NLP-Project/Sentiment-Analysis-Experiments/Dataset/{lang}_train.csv"
test_file = f"/kaggle/working/CSE556-NLP-Project/Sentiment-Analysis-Experiments/Dataset/{lang}_test.csv"

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

## Custom Dataset Class
- This class can be used for all Sentiment Analysis fine-tuning and zero shot tasks
- For multiple languages, combine data into one dataframe as the dataset class takes DATAFRAME as input

In [4]:
class MultilingualSentimentAnalysis_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        label_map = {"Negative": 0, "Positive": 1}
        
        label_value = self.dataframe.iloc[idx]["LABEL"]
        if pd.isna(label_value):
            label = 0
        else:
            label = label_map[label_value]
        
        
        input_text = self.dataframe.iloc[idx]["INDIC REVIEW"]

        # Tokenize
        encoding = self.tokenizer.encode_plus(
            input_text, None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

In [5]:
from transformers import AutoTokenizer, AutoModel
# Get the model name
model_name = "ai4bharat/indic-bert"
# Initialize IndicBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Initialise dataset instances
train_dataset = MultilingualSentimentAnalysis_Dataset(train_df, tokenizer)
dev_dataset = MultilingualSentimentAnalysis_Dataset(test_df, tokenizer)

# Initialise the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

In [7]:
import torch
import torch.nn as nn
from transformers import AutoModel

class MBertForSentimentAnalysis(nn.Module):
    def __init__(self, freeze_bert=False):
        super(MBertForSentimentAnalysis, self).__init__()

        # Load mBERT model and tokenizer
        self.model_name = "ai4bharat/indic-bert"
        self.mbert = AutoModel.from_pretrained(self.model_name)

        # Add a batch normalization layer
        self.batch_norm = nn.BatchNorm1d(self.mbert.config.hidden_size)
        
        # Add a linear layer for classification
        self.classification = nn.Linear(self.mbert.config.hidden_size, 2)

        # Option to freeze MBERT layers to prevent them from being updated during training
        if freeze_bert:
            for param in self.mbert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        # Get the output from BERT model
        _, pooled_outputs = self.mbert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)

        # Pass output through batch normalization layer
        pooled_outputs = self.batch_norm(pooled_outputs)
        
        # Pass output through linear layer
        out = self.classification(pooled_outputs)
        return out

In [8]:
model = MBertForSentimentAnalysis()
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)  # Move model to CUDA device if available
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU instead.")

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Using CUDA


In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
ce_loss = torch.nn.CrossEntropyLoss()

num_epochs = 4

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training
    model.train()  
    train_loss = 0
    for batch in train_dataloader:
        # Forward pass
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
        loss = ce_loss(outputs, inputs['labels'].long())

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()

    # Validation
    model.eval()  
    val_loss = 0
    with torch.no_grad():  # No need to compute gradients during validation
        for batch in dev_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            loss = ce_loss(outputs, inputs['labels'].long())
            val_loss += loss.item()

    # Calculate average losses
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(dev_dataloader)

    # Append the losses for plotting
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

Epoch 1/4, Train Loss: 0.6909, Validation Loss: 0.6870
Epoch 2/4, Train Loss: 0.6071, Validation Loss: 0.5667
Epoch 3/4, Train Loss: 0.4534, Validation Loss: 0.6148
Epoch 4/4, Train Loss: 0.3387, Validation Loss: 0.5320


### Evaluation of Fine-Tuned Model
- Evaluate fine-tuned model on test dataset of each language

In [10]:
# Load datasets for each language
languages = ["hi", "mr", "bn", "ta", "te"]

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

with torch.no_grad():
    for lang in languages:
        test_file = f"/kaggle/working/CSE556-NLP-Project/Sentiment-Analysis-Experiments/Dataset/{lang}_test.csv"
        test_df = pd.read_csv(test_file)
        test_dataloader = DataLoader(MultilingualSentimentAnalysis_Dataset(test_df, tokenizer), batch_size=16, shuffle=True)
        
        # Make list for predicted labels and ground truth labels
        predicted_labels = []
        labels = []

        # Perform inference
        for batch in test_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            predicted_labels.extend(torch.argmax(outputs, dim=1).tolist())
            labels.extend(inputs['labels'].long().tolist())
        

        # Print results for a particular language
        print(f"RESULTS FOR {lang}")
        print()
        # Calculate accuracy
        accuracy = accuracy_score(labels, predicted_labels) 
        print(f'Accuracy: {accuracy:.4f}')

        # Calculate F1-score
        weighted_f1_score = f1_score(labels, predicted_labels, average='weighted')
        macro_f1_score = f1_score(labels, predicted_labels, average='macro')
        print(f'Weighted F1-score: {weighted_f1_score:.4f}')
        print(f'Macro F1-score: {macro_f1_score:.4f}')
        print()

        # Print classification report
        print("Classification Report")
        print(classification_report(labels, predicted_labels))
        print()
        # Print confusion matrix
        print("Confusion Matrix")
        print(confusion_matrix(labels, predicted_labels))
        print()
        print("----------*******************---------------")

RESULTS FOR hi

Accuracy: 0.5577
Weighted F1-score: 0.4595
Macro F1-score: 0.4501

Classification Report
              precision    recall  f1-score   support

           0       0.54      0.96      0.69        81
           1       0.75      0.12      0.21        75

    accuracy                           0.56       156
   macro avg       0.65      0.54      0.45       156
weighted avg       0.64      0.56      0.46       156


Confusion Matrix
[[78  3]
 [66  9]]

----------*******************---------------
RESULTS FOR mr

Accuracy: 0.5833
Weighted F1-score: 0.5209
Macro F1-score: 0.5138

Classification Report
              precision    recall  f1-score   support

           0       0.56      0.93      0.70        81
           1       0.73      0.21      0.33        75

    accuracy                           0.58       156
   macro avg       0.64      0.57      0.51       156
weighted avg       0.64      0.58      0.52       156


Confusion Matrix
[[75  6]
 [59 16]]

----------*****

In [11]:
# Save Model and Tokenizer
model_save_path = "IndicBERT_SA_Telugu_FineTune.pth"
torch.save(model.state_dict(), model_save_path)